# Turn a directory into something that can be easily visualized on a map!

In [22]:
import pandas as pd; import numpy as np; from multiprocessing import Pool, Manager; import numpy as np;
import fiona, shapely; from osgeo import ogr; from shapely.geometry import mapping, shape
import matplotlib.pyplot as plt; import seaborn as sns
import matplotlib, os, json, sys, time, datetime

In [23]:
input_directory  = "/data/chime/geo/matthew/brevard_zone_a_stage_2_temporal_clusters_with_home"
output_directory = "/data/www/chime/movement-derivation/matthew_brevard_zone_a_users"

In [24]:
users_in = sorted(os.listdir(input_directory))
users_in = [x for x in users_in if x != "temporal_clustered_user_meta.json"]
print("Found {0} users in {1}".format(len(users_in), input_directory))

Found 82 users in /data/chime/geo/matthew/brevard_zone_a_stage_2_temporal_clusters_with_home


In [41]:
def loader_function(args):
    uFile, path, q = args
    u = json.load(open(path+"/"+uFile,'r'))
    tweets = []
    for t in u['features']:
        if t['geometry']:
            t['properties']['geometry'] = shape(t['geometry'])
        t['properties']['date'] = pd.Timestamp(t['properties']['date'])
        tweets.append(t['properties'])
    
    df = pd.DataFrame(tweets)
    
    df = df.sort_values(by='date')
    
    first_date = df['date'][0]
    
    df['time'] = df['date'].apply(lambda x: datetime.datetime.strftime(x, '%Y-%m-%dT%H:%M:%SZ'))
    df['h']    = df['date'].apply(lambda x: int((x - first_date).total_seconds()/3600))
    df['m']    = df['date'].apply(lambda x: int((x - first_date).total_seconds()/60))
    if not q is None:
        q.put(1)
    return df

In [42]:
#debug
loader_function((users_in[0], input_directory, None))

,cluster,cluster_center,coords,date,day_cluster,geo_delta,geometry,home_cluster_id,speed,text,time_delta,tweet_id,uid,user,time,h,m
0,1.0,"{""type"": ""Point"", ""coordinates"": [-80.14031676...","[-80.14035502, 25.78909041]",2016-08-01 14:52:15+00:00,3,NaN,POINT (-80.14035502 25.78909041),1.0,NaN,Our killer happy hour continues today 1PM-7PM ...,NaN,760126000843919360,733478166,AbbeyBrewingMIA,2016-08-01T14:52:15Z,0,0
1,1.0,"{""type"": ""Point"", ""coordinates"": [-80.14031676...","[-80.14035502, 25.78909041]",2016-08-01 16:32:53+00:00,3,0.0,POINT (-80.14035502 25.78909041),NaN,0.0,Enjoying an immaculate IPA @abbeybrewingmia on...,6038.0,760151326621925381,733478166,AbbeyBrewingMIA,2016-08-01T16:32:53Z,1,100
2,1.0,"{""type"": ""Point"", ""coordinates"": [-80.14031676...","[-80.14035502, 25.78909041]",2016-08-01 21:07:56+00:00,5,0.0,POINT (-80.14035502 25.78909041),NaN,0.0,2-4-1 happy hour going down right now @abbeybr...,16503.0,760220543861809152,733478166,AbbeyBrewingMIA,2016-08-01T21:07:56Z,6,375
3,1.0,"{""type"": ""Point"", ""coordinates"": [-80.14031676...","[-80.14035502, 25.78909041]",2016-08-01 23:55:13+00:00,5,0.0,POINT (-80.14035502 25.78909041),NaN,0.0,Finally unwinding with a dram of fine Islay sc...,10037.0,760262643777212416,733478166,AbbeyBrewingMIA,2016-08-01T23:55:13Z,9,542
4,1.0,"{""type"": ""Point"", ""coordinates"": [-80.14031676...","[-80.14035502, 25.78909041]",2016-08-02 17:05:31+00:00,4,0.0,POINT (-80.14035502 25.78909041),NaN,0.0,Happy hours going down right now @abbeybrewing...,61818.0,760521924506181632,733478166,AbbeyBrewingMIA,2016-08-02T17:05:31Z,26,1573
5,1.0,"{""type"": ""Point"", ""coordinates"": [-80.14031676...","[-80.14035502, 25.78909041]",2016-08-02 18:29:47+00:00,4,0.0,POINT (-80.14035502 25.78909041),NaN,0.0,Our killer happy hour continues today 1PM-7PM ...,5056.0,760543132001927168,733478166,AbbeyBrewingMIA,2016-08-02T18:29:47Z,27,1657
6,1.0,"{""type"": ""Point"", ""coordinates"": [-80.14031676...","[-80.14035502, 25.78909041]",2016-08-03 13:58:17+00:00,3,0.0,POINT (-80.14035502 25.78909041),NaN,0.0,Happy hump day from all of us @abbeybrewingmia...,70110.0,760837193799720961,733478166,AbbeyBrewingMIA,2016-08-03T13:58:17Z,47,2826
7,1.0,"{""type"": ""Point"", ""coordinates"": [-80.14031676...","[-80.14035502, 25.78909041]",2016-08-03 14:04:05+00:00,3,0.0,POINT (-80.14035502 25.78909041),NaN,0.0,Happy hump day from all of us @abbeybrewingmia...,348.0,760838655602819072,733478166,AbbeyBrewingMIA,2016-08-03T14:04:05Z,47,2831
8,1.0,"{""type"": ""Point"", ""coordinates"": [-80.14031676...","[-80.14035502, 25.78909041]",2016-08-03 21:02:28+00:00,5,0.0,POINT (-80.14035502 25.78909041),NaN,0.0,An awesome line up today for whiskey Wednesda...,25103.0,760943942267465728,733478166,AbbeyBrewingMIA,2016-08-03T21:02:28Z,54,3250
9,1.0,"{""type"": ""Point"", ""coordinates"": [-80.14031676...","[-80.14035502, 25.78909041]",2016-08-04 21:14:50+00:00,5,0.0,POINT (-80.14035502 25.78909041),NaN,0.0,"Happy National IPA day, from all of us, @abbe...",87142.0,761309442356936704,733478166,AbbeyBrewingMIA,2016-08-04T21:14:50Z,78,4702


In [43]:
#Parallel runtime
p = Pool(30)
m = Manager()
q = m.Queue()

args = [(i, input_directory, q) for i in users_in]
result = p.map_async(loader_function, args)

# monitor loop
while True:
    if result.ready():
        break
    else:
        size = q.qsize()
        sys.stderr.write("\rProcessed: {0}, {1:.3g}%".format(size, size/len(args)*100))
        time.sleep(0.5)
sys.stderr.write("\rProcessed: {0}, {1:.3g}%".format(q.qsize(), q.qsize()/len(args)*100))
users = result.get()
p.close()

Processed: 82, 100%

In [44]:
users[0].tail()

,cluster,cluster_center,coords,date,day_cluster,geo_delta,geometry,home_cluster_id,speed,text,time_delta,tweet_id,uid,user,time,h,m
134,1.0,"{""type"": ""Point"", ""coordinates"": [-80.14031676...","[-80.14035502, 25.78909041]",2016-10-20 20:28:30+00:00,4,0.0,POINT (-80.14035502 25.78909041),NaN,0.0,Old Fashioned Thursday's are back @abbeybrewin...,84885.0,789201649516498944,733478166,AbbeyBrewingMIA,2016-10-20T20:28:30Z,1925,115536
135,1.0,"{""type"": ""Point"", ""coordinates"": [-80.14031676...","[-80.14035502, 25.78909041]",2016-10-21 12:22:56+00:00,2,0.0,POINT (-80.14035502 25.78909041),NaN,0.0,Hoppy Friday from all of us @abbeybrewingmia w...,57266.0,789441841783451648,733478166,AbbeyBrewingMIA,2016-10-21T12:22:56Z,1941,116490
136,1.0,"{""type"": ""Point"", ""coordinates"": [-80.14031676...","[-80.14035502, 25.78909041]",2016-10-21 21:51:40+00:00,5,0.0,POINT (-80.14035502 25.78909041),NaN,0.0,Just tapped this hop bomb @abbeybrewingmia and...,34124.0,789584967986655232,733478166,AbbeyBrewingMIA,2016-10-21T21:51:40Z,1950,117059
137,1.0,"{""type"": ""Point"", ""coordinates"": [-80.14031676...","[-80.14035502, 25.78909041]",2016-10-21 22:04:44+00:00,5,0.0,POINT (-80.14035502 25.78909041),NaN,0.0,"Second glass, flipside of the coaster, and sti...",784.0,789588254408531969,733478166,AbbeyBrewingMIA,2016-10-21T22:04:44Z,1951,117072
138,1.0,"{""type"": ""Point"", ""coordinates"": [-80.14031676...","[-80.14035502, 25.78909041]",2016-10-21 22:49:05+00:00,5,0.0,POINT (-80.14035502 25.78909041),NaN,0.0,"Civil society, rustic, Hoppy and beautiful, @a...",2661.0,789599416445497344,733478166,AbbeyBrewingMIA,2016-10-21T22:49:05Z,1951,117116


# Write `geojson` files for the web

In [45]:
if not os.path.exists(output_directory):
    os.mkdir(output_directory)

In [46]:
def write_geojson(args):
    u, output_directory, q = args
    file = u['user'][0].lower()
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in u.iterrows():
        if pd.notnull(row.geometry):
            geom = mapping(row.geometry)
        else:
            geom = None
        if pd.notnull(row.speed):
            speed = row.speed
        else:
            speed = None
        feature = {'type':'Feature',
                   'geometry':geom,
                   'properties':{'time':row['time'],
                                 'h':row['h'],
                                 'm':row['m'],
                                 'user':row['user'],
                                 'text':row['text'],
                                 's':speed,
                                 'c':row['cluster']
                                 }
            }
        geojson['features'].append(feature)
    
    with open(output_directory+"/"+file+".geojson",'w') as oFile:
        json.dump(geojson,oFile)
        
    if q is not None:
        q.put(1)
    return 1

In [47]:
#Parallel runtime
p = Pool(30)
m = Manager()
q = m.Queue()

args = [(i, output_directory, q) for i in users]
result = p.map_async(write_geojson, args)

# monitor loop
while True:
    if result.ready():
        break
    else:
        size = q.qsize()
        sys.stderr.write("\rProcessed: {0}, {1:.3g}%".format(size, size/len(args)*100))
        time.sleep(0.5)
sys.stderr.write("\rProcessed: {0}, {1:.3g}%".format(q.qsize(), q.qsize()/len(args)*100))
result.get()
p.close()

Processed: 82, 100%